In [ ]:
import requests
import pandas as pd
from requests.adapters import HTTPAdapter, Retry
import re
from io import StringIO
import time
import os

def setup_session():
    """Set up a requests session with retry functionality"""
    session = requests.Session()
    retry_strategy = Retry(
        total=5,
        backoff_factor=0.25,
        status_forcelist=[500, 502, 503, 504]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("https://", adapter)
    return session

def get_next_link(headers):
    """Extract next link from response headers for pagination"""
    if "Link" in headers:
        match = re.search(r'<(.+)>; rel="next"', headers["Link"])
        if match:
            return match.group(1)
    return None

def get_phosphorylated_proteins_mammals(session):
    """Fetch phosphorylated proteins data for mammals from UniProt"""
    base_url = "https://rest.uniprot.org/uniprotkb/search"
    
    query_params = {
        'query': 'taxonomy_id:40674 AND ft_mod_res:Phospho* AND reviewed:true',
        'format': 'tsv',
        'fields': 'accession,sequence,ft_mod_res',
        'size': 500
    }
    
    results = []
    url = base_url
    
    try:
        while url:
            response = session.get(url, params=query_params if url == base_url else None)
            response.raise_for_status()
            
            total_results = int(response.headers.get("x-total-results", 0))
            
            df = pd.read_csv(StringIO(response.text), sep='\t')
            results.append(df)
            
            print(f"Retrieved {sum(len(df) for df in results)} of {total_results} records")
            
            url = get_next_link(response.headers)
            time.sleep(1)
            
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching data: {e}")
        if results:
            return pd.concat(results, ignore_index=True)
        return None
    
    return pd.concat(results, ignore_index=True) if results else None

In [ ]:
def extract_phospho_positions_and_evidence(mod_res_string):
    """
    Extract phosphorylation positions along with PubMed/UniProtKB identifiers
    Returns sorted list of tuples (position, [evidence_ids])
    """
    # Split string into individual modifications
    mod_entries = re.split(r'(?=MOD_RES)', mod_res_string)
    positions_with_evidence = []
    
    for entry in mod_entries:
        entry = entry.strip()
        if not entry.startswith('MOD_RES'):
            continue
        
        # Extract position number
        pos_match = re.search(r'MOD_RES\s+(\d+)', entry)
        if not pos_match:
            continue
        
        position = int(pos_match.group(1))
        
        # Check if it's a phosphorylation
        if '/note="Phospho' not in entry:
            continue
        
        # Extract evidence identifiers (PubMed or UniProtKB)
        evidence_ids = []
        
        # Find all PubMed:XXXXX patterns
        pubmed_matches = re.findall(r'PubMed:([0-9]+)', entry)
        for pubmed_id in pubmed_matches:
            evidence_ids.append(f"PubMed:{pubmed_id}")
        
        # Find all UniProtKB:XXXXX patterns
        uniprotkb_matches = re.findall(r'UniProtKB:([A-Z0-9]+)', entry)
        for uniprotkb_id in uniprotkb_matches:
            evidence_ids.append(f"UniProtKB:{uniprotkb_id}")
        
        if evidence_ids:
            positions_with_evidence.append((position, evidence_ids))
    
    return sorted(positions_with_evidence, key=lambda x: x[0])

def process_data(df):
    """
    Process data into the desired format with additional column 
    containing phosphorylation identification sources
    """
    if df is None:
        return None
    
    # Change column names
    df.columns = ['ID', 'Sequence', 'MOD_RES']
    
    # Remove duplicates and rows with missing data
    df = df.dropna()
    df = df.drop_duplicates()
    
    # Process each row
    result_rows = []
    
    for _, row in df.iterrows():
        # Extract phosphorylation positions and their sources
        phospho_data = extract_phospho_positions_and_evidence(row['MOD_RES'])
        
        if phospho_data:  # Only if phosphorylation positions exist
            # Create lists of positions and corresponding sources
            positions = [str(pos) for pos, _ in phospho_data]
            
            # For each position, combine all sources into one value
            evidence_sources = []
            for _, sources in phospho_data:
                if sources:
                    # Take only the first identifier from the list for each position
                    evidence_sources.append(sources[0])
                else:
                    evidence_sources.append('')
            
            result_rows.append({
                'ID': row['ID'],
                'Sequence': row['Sequence'],
                'MOD_RES': ','.join(positions),
                'MOD_RES_Accession': ','.join(evidence_sources)
            })
    
    # Create DataFrame from results
    if result_rows:
        result_df = pd.DataFrame(result_rows)
        return result_df
    else:
        return None

In [ ]:
def analyze_phosphorylation_data(df):
    """Display statistics and sample data from processed phosphorylation dataset"""
    if df is None:
        return
    
    print("\nStatistics:")
    print(f"Number of unique proteins: {len(df)}")
    
    # Number of modifications per protein
    df['MOD_Count'] = df['MOD_RES'].str.count(',') + 1
    print("\nStatistics of phosphorylation modifications per protein:")
    print(df['MOD_Count'].describe())
    
    # Example of first few proteins
    print("\nSample data (first 3 proteins):")
    sample_data = df[['ID', 'MOD_RES', 'MOD_RES_Accession']].head(3)
    for _, row in sample_data.iterrows():
        print(f"ID: {row['ID']}")
        print(f"Phosphorylation positions: {row['MOD_RES']}")
        print(f"Identification sources: {row['MOD_RES_Accession']}")
        print()

def split_proteins_by_evidence(input_file):
    """
    Load CSV file with phosphorylated protein data and split each record
    into multiple records according to evidence sources.
    
    Args:
        input_file: Path to CSV file containing data from previous analysis
    
    Returns:
        DataFrame containing split data
    """
    # Load CSV file
    print(f"Loading file: {input_file}")
    df = pd.read_csv(input_file)
    
    print(f"Number of records before splitting: {len(df)}")
    
    # Check if required columns exist
    required_columns = ['ID', 'Sequence', 'MOD_RES', 'MOD_RES_Accession']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")
    
    # List to store new records
    new_records = []
    
    # Process each row
    for _, row in df.iterrows():
        id_val = row['ID']
        sequence = row['Sequence']
        mod_positions = row['MOD_RES'].split(',')
        mod_accessions = row['MOD_RES_Accession'].split(',')
        
        # Check if number of positions and accessions is the same
        if len(mod_positions) != len(mod_accessions):
            print(f"WARNING: Mismatch in number of positions and accessions for ID: {id_val}. Skipping.")
            continue
        
        # Group positions by evidence
        evidence_to_positions = {}
        
        for pos, acc in zip(mod_positions, mod_accessions):
            # Extract main evidence identifier (before first comma, if there are more)
            main_evidence = acc.split(',')[0].strip()
            
            if not main_evidence:
                continue
                
            if main_evidence not in evidence_to_positions:
                evidence_to_positions[main_evidence] = []
            
            evidence_to_positions[main_evidence].append(pos)
        
        # Create new record for each unique evidence
        for evidence, positions in evidence_to_positions.items():
            new_records.append({
                'ID': id_val,
                'Sequence': sequence,
                'MOD_RES': ','.join(positions),
                'MOD_RES_Accession': evidence,
                'Position_Count': len(positions)
            })
    
    # Create new DataFrame with split data
    result_df = pd.DataFrame(new_records)
    
    print(f"Number of records after splitting: {len(result_df)}")
    
    # Sort by ID and number of positions (descending)
    result_df = result_df.sort_values(['ID', 'Position_Count'], ascending=[True, False])
    
    return result_df

In [ ]:
def modify_sequence_at_positions(row):
    """
    Modify sequence by converting characters to lowercase at given positions.
    Positions in MOD_RES are numbered from 1, so we need to subtract 1 for indices.
    """
    sequence = list(row['Sequence'])
    # Convert position string to list of numbers (only if positions are non-empty)
    if pd.notna(row['MOD_RES']) and row['MOD_RES'].strip():
        positions = [int(pos) for pos in row['MOD_RES'].split(',')]
        
        # Change characters to lowercase at given positions (subtract 1 to adjust to 0-indexing)
        for pos in positions:
            idx = pos - 1  # convert position to index
            if 0 <= idx < len(sequence):  # check if index is valid
                sequence[idx] = sequence[idx].lower()
    
    return ''.join(sequence)

def process_modified_sequences(input_file, output_file):
    """
    Process the input file to create modified sequences with lowercase at phosphorylation sites
    and save to output file
    """
    # Check if file exists
    if not os.path.exists(input_file):
        print(f"ERROR: File {input_file} does not exist.")
        return
    
    try:
        # Load file
        print(f"Loading file: {input_file}")
        split_df = pd.read_csv(input_file)
        
        # Add new column with modified sequences
        print("Creating modified sequences...")
        split_df['Modified_Sequence'] = split_df.apply(modify_sequence_at_positions, axis=1)
        
        # Save result to CSV file
        split_df.to_csv(output_file, index=False)
        print(f"Data saved to file: {output_file}")
        
        # Display basic information
        print(f"\nNumber of records: {len(split_df)}")
        print(f"Number of unique proteins: {split_df['ID'].nunique()}")
        
        # Examples for several different proteins
        print("\nExamples of modified sequences:")
        sample_ids = split_df['ID'].drop_duplicates().head(3)
        sample_data = split_df[split_df['ID'].isin(sample_ids)].groupby('ID').head(1)
        
        for _, row in sample_data.iterrows():
            print(f"ID: {row['ID']}")
            print(f"Identification source: {row['MOD_RES_Accession']}")
            print(f"Phosphorylation positions: {row['MOD_RES']}")
            
            # Display fragment of sequence around phosphorylation sites
            mod_positions = [int(pos) for pos in row['MOD_RES'].split(',')]
            if mod_positions:
                # Find first phosphorylation position
                pos = mod_positions[0]
                start = max(0, pos - 10)
                end = min(len(row['Sequence']), pos + 10)
                
                original_fragment = row['Sequence'][start:end]
                modified_fragment = row['Modified_Sequence'][start:end]
                
                print(f"Original sequence fragment: ...{original_fragment}...")
                print(f"Modified sequence fragment: ...{modified_fragment}...")
            print()
        
    except Exception as e:
        print(f"Error occurred during data processing: {e}")

In [ ]:
def main_fetch_data():
    """Main function to fetch and process initial phosphorylation data"""
    print("Starting download of phosphorylated protein data for mammals...")
    
    session = setup_session()
    raw_df = get_phosphorylated_proteins_mammals(session)
    
    if raw_df is not None:
        processed_df = process_data(raw_df)
        
        if processed_df is not None:
            output_file = "phosphorylated_proteins_mammals_clean.csv"
            processed_df.to_csv(output_file, index=False)
            print(f"\nData saved to file: {output_file}")
            
            analyze_phosphorylation_data(processed_df)
    else:
        print("Failed to fetch data.")

def main_split_by_evidence():
    """Main function to split proteins by evidence sources"""
    # Path to input file
    input_file = "phosphorylated_proteins_mammals_clean.csv"
    
    # Check if file exists
    if not os.path.exists(input_file):
        print(f"ERROR: File {input_file} does not exist.")
        return
    
    try:
        # Split data by evidence
        split_df = split_proteins_by_evidence(input_file)
        
        # Path to output file
        output_file = "phosphorylated_proteins_mammals_split_by_evidence.csv"
        
        # Save result to CSV file
        split_df.to_csv(output_file, index=False)
        print(f"Data saved to file: {output_file}")
        
        # Display statistics
        print("\nStatistics:")
        print(f"Number of unique proteins: {split_df['ID'].nunique()}")
        print(f"Number of all evidence variants: {len(split_df)}")
        print("\nNumber of phosphorylation sites per evidence variant:")
        print(split_df['Position_Count'].describe())
        
    except Exception as e:
        print(f"Error occurred during data processing: {e}")

def main_create_modified_sequences():
    """Main function to create modified sequences"""
    # Path to input file with evidence variants
    input_file = "phosphorylated_proteins_mammals_split_by_evidence.csv"
    output_file = "phosphorylated_proteins_with_modified_sequences.csv"
    
    process_modified_sequences(input_file, output_file)

def main():
    """Main execution flow"""
    # Uncomment the function you want to run
    main_fetch_data()
    main_split_by_evidence()
    main_create_modified_sequences()

if __name__ == "__main__":
    main()